In [2]:
pip install requests dash

Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Start the Dash application
application = dash.Dash(__name__)

# Set up the API access parameters
API_ACCESS_KEY = "e28ce1fa755c967f48ce869fcc78d2d3"
FIXER_IO_URL = "http://data.fixer.io/api/latest"
REQUEST_PARAMS = {"access_key": API_ACCESS_KEY}

# Function to retrieve currency data from the API
def get_currency_data():
    try:
        api_response = requests.get(FIXER_IO_URL, params=REQUEST_PARAMS)
        exchange_data = api_response.json()
        return exchange_data
    except Exception as error:
        print("Error occurred during data retrieval:", error)
        return None

# Define the layout of the Dash application
application.layout = html.Div([
    html.H1("Live Currency Exchange Rates"),
    html.Div(id='currency-rate-display'),
    dcc.Interval(
        id='update-interval',
        interval=10*1000,  # Milliseconds
        n_intervals=0
    )
])

# Callback to update the displayed rates every 10 seconds
@application.callback(Output('currency-rate-display', 'children'),
                      [Input('update-interval', 'n_intervals')])
def refresh_currency_display(n):
    retrieved_data = get_currency_data()
    if retrieved_data:
        current_rates = retrieved_data.get('rates')
        if current_rates:
            return html.Div([
                html.P(f"{currency}: {rate}") for currency, rate in current_rates.items()
            ])
        return "Currency data is not available at this time."
    return "Unable to fetch data. Please verify API access and network connectivity."


if __name__ == '__main__':
    application.run_server(debug=True, use_reloader=False)
